## Experimento 4. 
## Predicción de la movilidad ocupacional al regreso del último viaje migratorio

Este experimento se propone tomando como referencia principal el artículo "Occupational Mobility among Returned Migrants in Latin America: A Comparative Analysis". En este experimento se prentende analizar el potencial de mobilidad ocupacional en personas migrantes entre sus 25 y 45 años de edad.

Selección y limpieza de datos a partir de las tablas <b>LIFE</b> y <b>COMMUNITY</b>.

El dataset se forma a partir de la combinación de ambas tablas. Los atributos a utilizar para el análisis de mobilidad ocupacional son los propuestos por el autor del artículo referenciado.

El autor propone etiquetar el nivel ocupacional de las personas migrantes y no migrantes a partir de lo incluído en el MMP. Según el trabajo citado, la etiqueta de clase puede tomar cuatro valores posibles que clasifican el cambio ocupacional entre los 25 y 45 años de edad de una persona:<br>
1. Upward (en una categoría ocupacional mayor)
1. Downward (en una categoría ocupacional menor)
1. No movement (permanece en la misma categoría)
1. Undetermined (Dato perdido en la ocupación inicial o final)

Para predecir la mobilidad ocupacional, en el artículo oríginal se realizó el cálculo de varios modelos de regresión logística multinomial.
En este trabajo se propone realizar la misma tarea utilizando algoritmos de Machine Learning Supervisados para clasificación.

<br>

---

Bibliotecas utilizadas

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing

Rutas de acceso y almacenamiento de los archivos generados

In [2]:
#Ruta para leer el archivo a procesar
path_to_data_core_files = os.path.join('../'*3, 'Bases de datos', 'Princeton', 'Mexican Migration Project (MMP)', 'CSV', 'core_files')

path_to_data_supplemental_files = os.path.join('../'*3, 'Bases de datos', 'Princeton', 'Mexican Migration Project (MMP)', 'CSV', 'supplemental_files')


#Ruta para almacenar el dataset procesado
path_to_store = os.path.join('..', 'datasets','movilidad_ocupacional')

Variables independientes a utilizar propuestas por el autor del artículo citado

In [3]:
person_id_features = ['commun','surveypl','surveyyr','hhnum', 'age']
ind_features = ['yrborn', 'educ', 'jchange' ,'jobdur','uscumexp']
occupational_features = ['occup']
trip_features = ['usyr1','ustrips','usdocl','usoccl','usexp']
commun_id_features = ['COMMUN']
context_features = ['METROCAT','MINX270','MINX280','MINX290','MINX200','MINX210'] #De qué año la proporción de personas ganando menos del doble del salario mínimo se debe considerar? (Promedio?)

Clasificación de ocupaciones de acuerdo a las categorías utilizadas en el paper citado

In [4]:
#Nonmanual jobs of high qualification (NMHQ)
#e.g. professionals, teachers and managers
NMHQ = []
NMHQ.extend(range(110,120)) #Professionals (110-119)
NMHQ.extend(range(130,140)) #Educators (130-139)
NMHQ.extend(range(210,220)) #Administrators and directors in both public and private sector (210-219)
NMHQ.extend(range(510,520)) #Manufacturing/repair supervisors (510-519)
NMHQ.extend(range(610,620)) #Service and administration supervisors (610-619)

#Nonmanual jobs of low qualification (NMLQ)
#e.g. Sales workers and personal services
NMLQ = []
NMLQ.extend(range(710,720)) #Sales workers (710-719)
NMLQ.extend(range(720,730)) #Ambulatory workers (720-729)
NMLQ.extend(range(810,820)) #Personal services workers in establishments (810-819)

#Manual jobs of high qualification (MHQ)
#e.g. Plumbers, electricians and technicians
MHQ = []
MHQ.extend(range(120,130)) #Technical workers (120-129)
MHQ.extend(range(140,150)) #Occupations in the arts, performances and sports (140-149)
MHQ.extend(range(520,530)) #Manufacturing/repair skilled workers (520-529)
MHQ.extend(range(530,540)) #Manufacturing/repair heavy equipment operators (530-539)
MHQ.extend(range(550,560)) #Transportation workers (550-559)
MHQ.extend(range(830,840)) #Protection services workers (830-839)

#Manual jobs of low qualification (MLQ)
#e.g. Factory workers, common laborers, farmworkers
MLQ = []
MLQ.extend(range(410,420)) #Agriculture, husbandry, forestry/fisheries workers (410-419)
MLQ.extend(range(540,550)) #Manufacturing/repair unskilled workers (540-549)
MLQ.extend(range(620,630)) #Administrative and support workers (620-629)
MLQ.extend([820]) #Domestic services workers (820)

#Unemployed
#Unemployed/Not in the labor force
UNEMPLOYED = [10,20,21,30,40,41,42,43,50,51,52,53,54,60,61,62,63,64,99]


Lectura de archivos del MMP

In [5]:
life_file = os.path.join(path_to_data_core_files, 'life174.csv')
commun_file = os.path.join(path_to_data_supplemental_files, 'commun174.csv')

#Lectura de los archivos
life = pd.read_csv(life_file, na_values=['9999',' '], usecols=person_id_features + ind_features + occupational_features + trip_features)
commun = pd.read_csv(commun_file, na_values=['9999',' '], usecols=commun_id_features + context_features)


In [6]:
life = life[life['surveyyr'] >= 1998]

Cálculo de variables propuestas por el autor del artículo citado

In [7]:
#Para los atributos que no se pueden calcular por contener valores desconocidos o perdidos, se les asigna valor nulo si es considerable imputarlo.
#De otro modo, se les asiga el valor de 0 para los numéricos y N/A para los categoricos.
#Atributos numéricos: 'education','labor_exp_age45','first_migration_age','total_trips','accumulated_us_experience','mean_proportion_low_income'
#Atributos categoricos: 'birth_cohort', 'age_us_experience', 'legal_status', 'latest_us_occupation', 'locality_type'


personas = life.groupby(["commun", "hhnum"]) #Agrupar personas por comun y hhnum para poder iterar sobre ellas

data = pd.DataFrame(columns=['birth_cohort','education','labor_exp_age45','first_migration_age','total_trips','legal_status','latest_us_occupation','accumulated_us_experience',
                             'mean_proportion_low_income','occ_age25','occ_age45'])

for _, persona in personas:
    p = {}
    
    #---------------------Cohorte de nacimiento (categorico)--------------------------------

    p['birth_cohort'] = persona['yrborn'].min()

    #Nivel educativo (numérico)
    educacion = persona['educ'].to_numpy()
    if np.all(educacion == 9999): #Si no se tiene información educativa en ningún año
        p['education'] = np.nan #Se asigna un valor nulo
    else: #Si se tiene información educativa
        for e in educacion: #Se busca el nivel educativo más alto
            max_educ = 0 #Se inicializa la variable del nivel educativo más alto
            if e != 9999: #Si se tiene información educativa
                if e > max_educ: #Si el nivel educativo es mayor al nivel educativo más alto
                    max_educ = e #Se actualiza el nivel educativo más alto
        p['education'] = max_educ #Se asigna el nivel educativo más alto

    
    #--------------------Años de experiencia laboral a los 45 años (numérico)----------------
    
    job_durs = persona[persona['age'] <= 45]['jobdur'].to_numpy() #Se seleccionan los trabajos de la persona hasta los 45 años
    labor_exp = 0 #Se inicializa la variable de la experiencia laboral

    if len(job_durs) == 0: #Si no se tiene información sobre la duración de ningún trabajo
        p['labor_exp_age45'] = np.nan #Se asigna un valor nulo
    else: #Si se tiene información sobre la duración de trabajos
        for j in job_durs: #Se suman los años de experiencia laboral
            if np.isnan(j): #Si no se tiene información sobre la duración de un trabajo
                labor_exp = labor_exp + 0 #El trabajo no se considera
            else: #Si se tiene información sobre la duración de un trabajo
                labor_exp = labor_exp + j #Se suma la duración del trabajo a la experiencia laboral

        p['labor_exp_age45'] = labor_exp / 12 #Se convierte la experiencia laboral a años

    #--------------Edad en la primera migración (numérico) ----------------

    if (persona['usyr1'] == 8888.0).any(): #Si no se tiene información sobre la primera migración
        #p['first_migration_age'] = 0 #Se asigna cero a la edad de la primera migración
        continue #Se pasa a la siguiente persona (la persona actual no es migrante)
    else: #Si se tiene información sobre la primera migración
        p['first_migration_age'] = persona['usyr1'].min() - persona['yrborn'].min() #Se calcula la edad de la primera migración
    
    #--------------Número total de viajes (numérico) ----------------
    if persona['ustrips'].max() == 8888: #Si no aplican los viajes (nacido en Estados Unidos)
        p['total_trips'] = 0 #Se asigna un valor nulo
    else:
        p['total_trips'] = persona['ustrips'].max() #Se considera el número total de viajes

    #--------------Posesión de documentos durante el último viaje migratorio (categórico) ----------------
    #Dos categorías: 1) Con documentos y 2) Sin documentos
    if(persona['usdocl'].max() == 8888 or persona['usdocl'].max() == 9999): #Si no se tiene información sobre la posesión de documentos
        p['legal_status'] = 'N/A' #Se asigna un valor nulo
    elif(persona['usdocl'].max() == 8): #Si la persona no tenía documentos
        p['legal_status'] = 'WD' #Se considera que la persona no tenía documentos
    else: #Si la persona tenía documentos
        p['legal_status'] = 'WOD' #Se considera que la persona tenía documentos

    #--------------Ocupación durante el último viaje a Estados Unidos (categórico) ----------------
    #Tres categorías: 1) Manual, 2) Agraria y 3) Otras
    if(persona['usoccl'] == 8888.0).any() or (persona['usoccl'] == 9999.0).any(): #Si no se tiene información sobre la ocupación
        p['latest_us_occupation'] = 'N/A' #Se asigna un valor nulo
    else: #Si se tiene información sobre la ocupación
        if persona['usoccl'].max() in range(410,420): #Si la ocupación es agraria
            p['latest_us_occupation'] = 'Agrarian' #Se considera que la ocupación es agraria
        elif persona['usoccl'].max() in MHQ or persona['usoccl'].max() in MLQ and persona['usoccl'].max(): #Si la ocupación es manual
            p['latest_us_occupation'] = 'Manual' #Se considera que la ocupación es manual
        else: #Si la ocupación no es agraria ni manual
            p['latest_us_occupation'] = 'Other' #Se considera que la ocupación es otra

    #--------------Experiencia acumulada en Estados Unidos en meses (numérico) ----------------
    if (np.isnan(persona['usexp'].max())): #Si no se tiene información sobre la experiencia acumulada
        p['accumulated_us_experience'] = np.nan #Se asigna un valor nulo para imputar
    elif (persona['usexp'].max() == 8888): #Si es una persona nacida en Estados Unidos o nunca ha migrado
        p['accumulated_us_experience'] = 0 #La experiencia acumulada es cero
    else: #Si se tiene información sobre la experiencia acumulada y no es una persona nacida en Estados Unidos
        p['accumulated_us_experience']=(persona['usexp'].max()) #Se considera la experiencia acumulada en meses

    c = commun[commun['COMMUN'] == persona['commun'].max()] #Selección de la comunidad de la persona


    #---------------Proporción de personas ganando menos del doble del salario mínimo (numérico) ----------------
    #Se calcula el promedio de las proporciones de personas ganando menos del doble del salario mínimo en las décadas de 1970 y 1980, 1990, 2000 y 2010 en la comunidad seleccionada
    props = np.array([]) #Se inicializa un arreglo para almacenar las proporciones

    #Década de 1970
    if(c['MINX270'].item() == 8888 or np.isnan(c['MINX270'].item())): #Si no se tiene información
        #props = np.append(props,0) #Se asigna cero
        pass
    else: #Si se tiene información
        props = np.append(props,c['MINX270']) #Se considera la proporción reportada

    #Década de 1980
    if(c['MINX280'].item() == 8888 or np.isnan(c['MINX280'].item())): #Si no se tiene información
        #props = np.append(props,0) #Se asigna cero
        pass
    else: #Si se tiene información
        props = np.append(props,c['MINX280']) #Se considera la proporción reportada

    #Década de 1990
    if(not (c['MINX290'].item() == 8888 or np.isnan(c['MINX290'].item()))): #Si no se tiene información
        #props = np.append(props,0) #Se asigna cero
        pass
    else: #Si se tiene información
        props = np.append(props,c['MINX290']) #Se considera la proporción reportada

    #Década de 2000
    if(c['MINX200'].item() == 8888 or np.isnan(c['MINX200'].item())): #Si no se tiene información
        #props = np.append(props,0) #Se asigna cero
        pass
    else: #Si se tiene información
        props = np.append(props,c['MINX200']) #Se considera la proporción reportada

    #Década de 2010
    if(c['MINX210'].item() == 8888 or np.isnan(c['MINX210'].item())): #Si no se tiene información
        #props = np.append(props,0) #Se asigna cero
        pass
    else: #Si se tiene información
        props = np.append(props,c['MINX210']) #Se considera la proporción reportada

    p['mean_proportion_low_income'] = np.mean(props) #Se calcula el promedio de las proporciones

    #Ocupación a los 25 y 45 años
    if (persona['age'] == 25).any(): #Si se tiene un registro de la persona a los 25 años
        p['occ_age25'] = persona[persona['age'] == 25]['occup'].item() #Se considera la ocupación a los 25 años
    else: #Si no se tiene un registro de la persona a los 25 años
        p['occ_age25'] = np.nan #Se asigna un valor nulo
        
    if (persona['age'] == 45).any(): #Si se tiene un registro de la persona a los 45 años
        p['occ_age45'] = persona[persona['age'] == 45]['occup'].item() #Se considera la ocupación a los 45 años
    else: #Si no se tiene un registro de la persona a los 45 años
        p['occ_age45'] = np.nan #Se asigna un valor nulo

    #Se convierte el diccionario en un DataFrame y se concatena con el DataFrame principal
    p = pd.DataFrame(p, index=[0]) #Se convierte el diccionario en un DataFrame
    data = pd.concat([data,p], ignore_index=True) #Se concatena el DataFrame con el principal 

/tmp/ipykernel_221518/924288226.py:179: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data,p], ignore_index=True) #Se concatena el DataFrame con el principal


NOTA: REMOVER PERSONAS CON MÀS DE 45 AÑOS DE EXPERIENCIA? NO TIENE SENTIDO TENER MÁS DE 45 AÑOS DE EXPERIENCIA LABORAL A LA EDAD DE 45 AÑOS

Etiquetado de ocupaciones de acuerdo a las categorías definidas por el autor

In [8]:
data['occ_age25'] = data['occ_age25'].apply(lambda x: 
                                            'Unemployed' if x in UNEMPLOYED 
                                            else 'NMHQ' if x in NMHQ 
                                            else 'NMLQ' if x in NMLQ
                                            else 'MHQ' if x in MHQ 
                                            else 'MLQ' if x in MLQ
                                            else 'N/A')

data['occ_age45'] = data['occ_age45'].apply(lambda x: 
                                            'Unemployed' if x in UNEMPLOYED
                                            else 'NMHQ' if x in NMHQ
                                            else 'NMLQ' if x in NMLQ
                                            else 'MHQ' if x in MHQ
                                            else 'MLQ' if x in MLQ
                                            else 'N/A')

In [9]:
data.to_csv(os.path.join(path_to_store, 'movilidad_ocupacional_unlabeled.csv'), index=False)
data

,birth_cohort,education,labor_exp_age45,first_migration_age,total_trips,legal_status,latest_us_occupation,accumulated_us_experience,mean_proportion_low_income,occ_age25,occ_age45
0,1993,9.0,9.500000,22.0,5,WOD,Agrarian,30.0,0.186033,MLQ,N/A
1,1992,9.0,8.500000,19.0,9,WOD,Agrarian,54.0,0.186033,MLQ,N/A
2,1992,9.0,10.166667,23.0,5,WOD,Agrarian,26.0,0.186033,MLQ,N/A
3,1988,9.0,15.500000,19.0,3,WOD,Agrarian,66.0,0.186033,MLQ,N/A
4,1979,11.0,23.500000,18.0,3,WD,Manual,258.0,0.186033,MHQ,N/A
...,...,...,...,...,...,...,...,...,...,...,...
4825,1941,4.0,35.000000,63.0,1,WD,Agrarian,60.0,0.079412,MLQ,MLQ
4826,1985,6.0,22.000000,31.0,1,WD,Other,24.0,0.079412,MLQ,N/A
4827,1970,6.0,34.000000,15.0,2,WD,Other,96.0,0.079412,MLQ,MLQ
4828,1982,9.0,21.000000,12.0,4,WD,Other,192.0,0.079412,MLQ,N/A


Etiquetado de mobilidad ocupacional de acuerdo al criterio descrito por el autor

In [10]:
def label_mobility(data):
    ranks = {'NMHQ': 5, 'NMLQ': 4, 'MHQ': 3, 'MLQ': 2, 'Unemployed': 1, 'N/A': 0}

    if data['occ_age25'] == 'N/A' or data['occ_age45'] == 'N/A':
        return 'undetermined'
    elif ranks[data['occ_age25']] == ranks[data['occ_age45']]:
        return 'no_movement'
    elif ranks[data['occ_age25']] > ranks[data['occ_age45']]:
        return 'downward'
    else:
        return 'upward'

data['label'] = data.apply(label_mobility, axis=1)

In [11]:
data.to_csv(os.path.join(path_to_store, 'movilidad_ocupacional_labeled.csv'), index=False)

In [12]:
data.drop(['occ_age25','occ_age45'], axis=1, inplace=True)
data.to_csv(os.path.join(path_to_store, 'movilidad_ocupacional_no_occupations_labeled.csv'), index=False)
data

,birth_cohort,education,labor_exp_age45,first_migration_age,total_trips,legal_status,latest_us_occupation,accumulated_us_experience,mean_proportion_low_income,label
0,1993,9.0,9.500000,22.0,5,WOD,Agrarian,30.0,0.186033,undetermined
1,1992,9.0,8.500000,19.0,9,WOD,Agrarian,54.0,0.186033,undetermined
2,1992,9.0,10.166667,23.0,5,WOD,Agrarian,26.0,0.186033,undetermined
3,1988,9.0,15.500000,19.0,3,WOD,Agrarian,66.0,0.186033,undetermined
4,1979,11.0,23.500000,18.0,3,WD,Manual,258.0,0.186033,undetermined
...,...,...,...,...,...,...,...,...,...,...
4825,1941,4.0,35.000000,63.0,1,WD,Agrarian,60.0,0.079412,no_movement
4826,1985,6.0,22.000000,31.0,1,WD,Other,24.0,0.079412,undetermined
4827,1970,6.0,34.000000,15.0,2,WD,Other,96.0,0.079412,no_movement
4828,1982,9.0,21.000000,12.0,4,WD,Other,192.0,0.079412,undetermined


In [13]:
print(f'Cardinalidad de las clases:\n{data['label'].value_counts()}')
print(f'Total: {len(data)}')

Cardinalidad de las clases:
label
undetermined    2490
no_movement     1486
upward           564
downward         290
Name: count, dtype: int64
Total: 4830


Codificación de datos categóricos a numéricos

In [14]:
#Eliminación de las instancias con etiqueta 'undetermined', ya que no aportan información
data = data[data['label'] != 'undetermined']
#Eliminación de las instancias con etiqueta 'no_movement' para llevar a cabo un problema de clasificación binaria
#data = data[data['label'] != 'no_movement']
data.reset_index(drop=True, inplace=True) #Reiniciar los índices (!)
#cat_atts = ['birth_cohort','age_us_experience', 'legal_status', 'latest_us_occupation', 'locality_type'] #Atributos categóricos
cat_atts = ['birth_cohort', 'legal_status', 'latest_us_occupation'] #Atributos categóricos
num_atts = ['education','labor_exp_age45','first_migration_age','total_trips','accumulated_us_experience','mean_proportion_low_income'] #Atributos numéricos

cat_X = data.drop('label', axis=1)[cat_atts] #Datos categóricos
num_X = data.drop('label', axis=1)[num_atts] #Datos numéricos

y = data['label'] #Etiquetas

ord_enc = preprocessing.OrdinalEncoder() #Codificador ordinal
lab_enc = preprocessing.LabelEncoder() #Codificador de etiquetas

cat_X = pd.DataFrame(ord_enc.fit_transform(cat_X), columns=cat_atts) #Codificación de los datos categóricos
num_X = num_X.astype('float64') #Conversión de los datos numéricos a flotantes
y = pd.DataFrame(lab_enc.fit_transform(y), columns=['label']) #Codificación de las etiquetas

X = pd.concat([cat_X, num_X], axis=1) #Datos codificados en un solo DataFrame

data = pd.concat([X,y], axis=1) #Datos codificados y etiquetas en un solo DataFrame

In [15]:
data.to_csv(os.path.join(path_to_store, 'movilidad_ocupacional_encoded_labeled_THREE_CLASSES.csv'), index=False)
data

,birth_cohort,legal_status,latest_us_occupation,education,labor_exp_age45,first_migration_age,total_trips,accumulated_us_experience,mean_proportion_low_income,label
0,40.0,1.0,2.0,4.0,23.666667,67.0,4.0,9.0,0.186033,2
1,60.0,1.0,0.0,4.0,32.833333,19.0,3.0,380.0,0.186033,0
2,65.0,1.0,0.0,5.0,36.000000,29.0,1.0,204.0,0.186033,1
3,23.0,0.0,0.0,0.0,29.416667,20.0,1.0,5.0,0.322872,1
4,34.0,0.0,1.0,0.0,29.083333,36.0,1.0,4.0,0.322872,1
...,...,...,...,...,...,...,...,...,...,...
2335,58.0,0.0,2.0,3.0,36.000000,20.0,3.0,84.0,0.079412,0
2336,51.0,0.0,2.0,3.0,37.000000,18.0,3.0,48.0,0.079412,1
2337,32.0,0.0,0.0,4.0,35.000000,63.0,1.0,60.0,0.079412,1
2338,61.0,0.0,2.0,6.0,34.000000,15.0,2.0,96.0,0.079412,1


In [16]:
data.isna().sum() #Verificación de datos perdidos

birth_cohort                  0
legal_status                  0
latest_us_occupation          0
education                     0
labor_exp_age45               0
first_migration_age           0
total_trips                   0
accumulated_us_experience     1
mean_proportion_low_income    0
label                         0
dtype: int64

Etiquetas de clase

In [17]:
dict(zip(lab_enc.classes_,lab_enc.transform(lab_enc.classes_))) #Etiquetas de clase

{'downward': 0, 'no_movement': 1, 'upward': 2}

Tipos de dato

In [18]:
data.dtypes

birth_cohort                  float64
legal_status                  float64
latest_us_occupation          float64
education                     float64
labor_exp_age45               float64
first_migration_age           float64
total_trips                   float64
accumulated_us_experience     float64
mean_proportion_low_income    float64
label                           int64
dtype: object